In [7]:
import numpy as np

In [8]:
import pandas as pd

In [29]:
import os, sys

In [14]:
!mkdir /scratch/sv1239/projects/chen/raw/

In [15]:
!mv /scratch/sv1239/Archive.zip /scratch/sv1239/projects/chen/raw/

In [16]:
!ls -a /scratch/sv1239/projects/chen/raw/

.  ..  Archive.zip


In [17]:
!cd /scratch/sv1239/projects/chen/raw/

In [ ]:
!unzip /scratch/sv1239/projects/chen/raw/Archive.zip

Archive:  /scratch/sv1239/projects/chen/raw/Archive.zip
   creating: 1880_complete/
   creating: 1880_complete/txt/
  inflating: 1880_complete/txt/X19113QNB5G0.txt  
  inflating: 1880_complete/txt/X19127QNB5G0.txt  
  inflating: 1880_complete/txt/X1913RQNB5G0.txt  
  inflating: 1880_complete/txt/XALNA1.txt  
  inflating: 1880_complete/txt/XARHQVQNB5G0.txt  
  inflating: 1880_complete/txt/XB0OHBQNB5G0.txt  
  inflating: 1880_complete/txt/XB0OSRQNB5G0.txt  
  inflating: 1880_complete/txt/XB0OVRQNB5G0.txt  
  inflating: 1880_complete/txt/XB0OVVQNB5G0.txt  
  inflating: 1880_complete/txt/XB0P2VQNB5G0.txt  
  inflating: 1880_complete/txt/XB0P37QNB5G0.txt  
  inflating: 1880_complete/txt/XB0P5FQNB5G0.txt  
  inflating: 1880_complete/txt/XB0P5JQNB5G0.txt  
  inflating: 1880_complete/txt/XB0P6BQNB5G0.txt  
  inflating: 1880_complete/txt/XB0P8VQNB5G0.txt  
  inflating: 1880_complete/txt/XB0PAFQNB5G0.txt  
  inflating: 1880_complete/txt/XB0PANQNB5G0.txt  
  inflating: 1880_complete/txt/XB0PH7QNB

In [28]:
!pwd

/home/sv1239/chen


In [22]:
!ls /scratch/sv1239/projects/chen/raw/

Archive.zip


In [37]:
[folder for folder in os.listdir(folder_path)]

['1881_complete',
 '1886_complete',
 '1889_complete',
 '1890_complete',
 '1880_complete',
 '1888_complete',
 '1887_complete',
 '1884_complete',
 '1883_complete',
 '.ipynb_checkpoints',
 '1885_complete',
 'Untitled.ipynb',
 '1882_complete']

In [108]:
def read_data(folder_path):
    corpus = ""
    for folder in os.listdir(folder_path):
        for file_name in os.listdir(folder_path+"/"+folder+"/"):
            if ".txt" in file_name:
                with open(folder_path+folder+"/"+file_name, 'r') as f:
                    corpus+=f.read()
        
    print corpus[:10]
    return corpus

In [109]:
%%time

folder_path = "/scratch/sv1239/projects/chen/raw/"
# os.chdir(folder_path)

corpus = read_data(folder_path)


 ACHESON,
CPU times: user 439 ms, sys: 4.08 s, total: 4.52 s
Wall time: 6.21 s


In [110]:
len(corpus)

94087200

In [121]:
corpus[:1000]

'\n ACHESON, J. This is a suit by the Torpedo Company, a corporation of the state of Delaware doing business in the state of Pennsylvania, against the incorporated borough of Clarendon, in Warren county, in the latter state, to restrain the enforcement against the plaintiff of an ordinance of the borough, enacted April 24, 1882, which declares it to be unlawful for any person to "store, house, convey, carry, or have in his or her possession," within the borough limits, any nitro-glycerine, (except enough to "shoot" any oil well in the borough, on payment of a license fee of $10,) under a penalty of not less than $50, nor more than $100, for each offense, upon conviction before the burgess or a justice of the peace. The proper operation of oil wells, it seems, requires that torpedoes containing nitro-glycerine be exploded from time to time in the wells. The plaintiff has established works for the manufacture of nitro-glycerine in the county of Warren, nine miles from Clarendon, and on t

In [112]:
import string, re

In [128]:
text = corpus.translate(None, string.punctuation).replace("\n", " ") #remove punctuations, \n's
text = re.sub(r'[^\x00-\x7F]+',' ', text) #remove non-ASCII chars
text = re.sub( '\s+', ' ', text).lstrip().rstrip() #remove extra and trailing spaces

In [129]:
text[:10000]

'ACHESON J This is a suit by the Torpedo Company a corporation of the state of Delaware doing business in the state of Pennsylvania against the incorporated borough of Clarendon in Warren county in the latter state to restrain the enforcement against the plaintiff of an ordinance of the borough enacted April 24 1882 which declares it to be unlawful for any person to store house convey carry or have in his or her possession within the borough limits any nitroglycerine except enough to shoot any oil well in the borough on payment of a license fee of 10 under a penalty of not less than 50 nor more than 100 for each offense upon conviction before the burgess or a justice of the peace The proper operation of oil wells it seems requires that torpedoes containing nitroglycerine be exploded from time to time in the wells The plaintiff has established works for the manufacture of nitroglycerine in the county of Warren nine miles from Clarendon and on the opposite side of the borough there has b

In [135]:
len(text.split())

16490758

In [130]:
with open('circuit_corpus_1880_1890.txt', 'w') as f:
    f.write(text)

In [134]:
!wc -w circuit_corpus_1880_1890.txt

16490758 circuit_corpus_1880_1890.txt


###TODO

- Remove unicode chars - DONE
- Replace numericals/dates with NN/NN/NNNN or NN.NNNN etc. - TBD. Check how people typically do it. Can also just remove all numbers.
- GloVe vectors for 10 years (1880-1890) - DONE
- GloVe vectors for entirety of dataset (~130 years) - TBD
- Add bias tests comparing pre-trained and our GloVe vectors - TBD

Note:

- While above process works for 10 years worth of data (32 MB), it may be a problem for larger datasets (> 100MB), since we are combining all the documents into one single document/string (separated by spaces), and creating a huge text file might lead to OOM issues. Can remedy by increasing memory in HPC (and specify RAM in run.sh below), but also look up how GloVe typically trained on huge corpuses, i.e. can we train in batches?
- Once above process (corpus generation) is complete, do the following:
- Go to https://github.com/stanfordnlp/GloVe and read the README
- Clone the GloVe repo, try running the demo.sh
- Copy the corpus file to the glove/ folder, and create a modified version of demo.sh, say, run.sh, which uses this file. Run.sh is provided in this repo.
- Modify the params in run.sh accordingly (eg. dimension of vectors, threads etc.)

